In [1]:
import os
!mkdir flaskwebapp

mkdir: flaskwebapp: File exists


In [2]:
cd ./flaskwebapp/

/Users/shiwang/Desktop/Github/cloud_learn/flaskwebapp


## define docker File

In [11]:
%%writefile Dockerfile

# Use an official Python runtime as a parent image
FROM python:3.6-slim

# Copy the current directory contents into the container at /app. Different from mount
ADD . /app

# Set the working directory to /app
WORKDIR /app

# Install any needed packages specified in requirements.txt
RUN pip install --trusted-host pypi.python.org -r requirements.txt

# Make port 80 available to the world outside this container
EXPOSE 80

# Run app.py when the container launches
CMD ["python", "app.py"]

Writing Dockerfile


## define `app.py` file

In [12]:
%%writefile app.py
import os
from flask import Flask
from flask import jsonify

app = Flask(__name__)

@app.route('/api/test')
def test():
    return jsonify(result = "ok")

if __name__ == '__main__':
    port = 80
    app.run(host='0.0.0.0',port=int(port))

Writing app.py


## define `app.yaml` file

In [31]:
%%writefile app.yaml
runtime: python27
api_version: 1
threadsafe: true

handlers:
- url: /.*
  script: app.app

Overwriting app.yaml


## Requirement file

In [25]:
%%writefile requirements.txt
Flask==0.12.2
flask-wtf

Overwriting requirements.txt


## Clear all container and images

In [77]:
!docker stop `docker ps -qa`
!docker rm `docker ps -qa`

0937dd9de2b6
0937dd9de2b6


In [65]:
!docker image rm -f $(docker image ls -a -q)

"docker image rm" requires at least 1 argument.
See 'docker image rm --help'.

Usage:  docker image rm [OPTIONS] IMAGE [IMAGE...] [flags]

Remove one or more images


## Build docker

In [66]:
!docker build -t testtest .

Sending build context to Docker daemon  5.632kB
Step 1/6 : FROM python:3.6-slim
3.6-slim: Pulling from library/python

bbb4ea60: Pulling fs layer 
216446be: Pulling fs layer 
37d373e1: Pulling fs layer 
0ee5721c: Pulling fs layer 
Digest: sha256:927b7bdc77aeca7dc7e6db256e6ef87dcba274ead80124ec6393e19f08001dfb
Status: Downloaded newer image for python:3.6-slim
 ---> 3197d7cc706b
Step 2/6 : ADD . /app
 ---> 061dad00854f
Step 3/6 : WORKDIR /app
Removing intermediate container 2bc57ede7012
 ---> ba1f46a23ee4
Step 4/6 : RUN pip install --trusted-host pypi.python.org -r requirements.txt
 ---> Running in 7ce48f11ff4f
  Running setup.py bdist_wheel for itsdangerous: started
  Running setup.py bdist_wheel for itsdangerous: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/2c/4a/61/5599631c1554768c6290b08c02c72d7317910374ca602ff1e5
  Running setup.py bdist_wheel for MarkupSafe: started
  Running setup.py bdist_wheel for MarkupSafe: finished with status 'done'
  Stored in

## View image list

In [67]:
!docker image ls

REPOSITORY          TAG                 IMAGE ID            CREATED              SIZE
testtest            latest              295228676c27        About a minute ago   149MB
python              3.6-slim            3197d7cc706b        7 days ago           138MB


## Run docker on local

In [68]:
# -d: background
# -v: mount local to docker
# -p: localhost->docker port
# -it: interactivae
!docker run -d -it -v ~/Desktop/Github/cloud_learn/flaskwebapp:/appapp -p 5000:80 testtest

2b0a165aa466367d2d1a17009823cfed8f1b3be2f39384f537e39c9986c735f6


View running container

In [69]:
!docker ps

CONTAINER ID        IMAGE               COMMAND             CREATED                  STATUS              PORTS                  NAMES
2b0a165aa466        testtest            "python app.py"     Less than a second ago   Up 2 seconds        0.0.0.0:5000->80/tcp   hopeful_payne


## View results

In [70]:
!curl http://0.0.0.0:5000/api/test

{"result":"ok"}


http://localhost:5000/api/test

## Optional: run commands in docker

---
docker exec -it 542061972ffb /bin/bash

---

## push image to remote

---

!docker login

---

In [ ]:
!docker tag testtest kingstone010148/test_images
!docker push kingstone010148/test_images

## run remote image

In [79]:
!docker run -d -p 5000:80 kingstone010148/test_images

28be54cd865b98d863b19177ea57c1307cb0284dd132c20d83bebb056c80147b


In [97]:
!curl http://0.0.0.0:5000/api/test

curl: (7) Failed to connect to 0.0.0.0 port 5000: Connection refused


Next step coudl be deploy a containerized web application on a Kubernetes Engine cluster 

https://cloud.google.com/kubernetes-engine/docs/tutorials/hello-app